Give the average sentiment PER CANTON or per city (more precise is imposible)

In [1]:
import numpy as np
import pandas as pd

In [2]:
twitter=pd.read_json("harvest3r_twitter_data_12-01_0.json");

# File Merger (on the server)!

In [ ]:
import os, os.path
files=[name for name in os.listdir('.') if os.path.isfile(name)];
AllTwitter=[];
AllInstagram=[];
AllNews=[];
a=0;
#Add Loop "for" to go in different directory (january to october)
for i in ['january','february','march','april','mai','june','july','august','september','october']:
    files=[name for name in os.listdir('./'+i+"/") if os.path.isfile(name)]; #Change the path accordingly
    for i in files:
        if files[a].find('twitter')>0:
            try:
                twitter=pd.read_json(i);
                AllTwitter.append(twitter);
            except ValueError:
                print ("JSON object issue");
        elif files[a].find('instagram')>0:
            try:
                instagram=pd.read_json(i);
                AllInstagram.append(instagram);
            except ValueError:
                print ("JSON object issue");
        elif files[a].find('news')>0:
            try:
                news=pd.read_json(i);
                AllNews.append(news);
            except ValueError:
                print ("JSON object issue");
        a=a+1;

#To transform into DataFrame
AllTwitter=pd.concat(AllTwitter, ignore_index=True); #To transform into DataFrame
AllInstagram=pd.concat(AllInstagram, ignore_index=True);
AllNews=pd.concat(AllNews, ignore_index=True)

# File merger (locally)!

In [64]:
import os, os.path
files=[name for name in os.listdir('.') if os.path.isfile(name)];
AllTwitter=[];
AllInstagram=[];
AllNews=[];
a=0;

files=[name for name in os.listdir('.') if os.path.isfile(name)]; 
for i in files:
    if files[a].find('twitter')>0:
        try:
            twitter=pd.read_json(i);
            AllTwitter.append(twitter);
        except ValueError:
            print ("JSON object issue");
    elif files[a].find('instagram')>0:
        try:
            instagram=pd.read_json(i);
            AllInstagram.append(instagram);
        except ValueError:
            print ("JSON object issue");
    elif files[a].find('news')>0:
        try:
            news=pd.read_json(i);
            AllNews.append(news);
        except ValueError:
            print ("JSON object issue");
    a=a+1;

#To transform into DataFrame
AllTwitter=pd.concat(AllTwitter, ignore_index=True); #To transform into DataFrame
AllInstagram=pd.concat(AllInstagram, ignore_index=True);
AllNews=pd.concat(AllNews, ignore_index=True)

JSON object issue


# Code to separate row with sentiment to row without + Write in positive.txt and negative.txt + delete stop_words (useless ones)


# I cannot concatenate, don't know why !

In [25]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from nltk.stem import PorterStemmer

def deleteContent(pfile):
    pfile.seek(0)
    pfile.truncate()

additional_stopwords=['.',',','/','(',')','[',']',':','@','\'','?','!','\"',';','&']
TwitterWithPos=[]
TwitterWithNeg=[]
TwitterWithout=[]
fPos = open('positive.txt', 'r+')
fNeg = open('negative.txt', 'r+')
fNeu = open('neutral.txt', 'r+')
#English
#fPosEN = open('positive.txt', 'r+')
#fNegEN = open('negative.txt', 'r+')
#fNeuEN = open('neutral.txt', 'r+')
#French
#fPosFR = open('positive.txt', 'r+')
#fNegFR = open('negative.txt', 'r+')
#fNeuFR = open('neutral.txt', 'r+')
#German
#fPosDE = open('positive.txt', 'r+')
#fNegDE = open('negative.txt', 'r+')
#fNeuDE = open('neutral.txt', 'r+')
#Italian
#fPosIT = open('positive.txt', 'r+')
#fNegIT = open('negative.txt', 'r+')
#fNeuIT = open('neutral.txt', 'r+')

deleteContent(fPos)
deleteContent(fNeg)
deleteContent(fNeu)
twitter['language'] = pd.Series(np.zeros(len(twitter)));

for i in range(0,400): #TO CHANGE !!! len(twitter)
    sentence=twitter.iloc[i]["_source"].get('main');
    language='None';
    try:
        language=detect(sentence); 
    except:
        e=1;
    twitter.ix[i,'language']=language;
    #FIND the language of the current Tweet
    if language=='en' or language=='de' or language=='it' or language=='fr': 
        langDict = {'en': 'english', 'de': 'german', 'it':'italian', 'fr':'french'} 
        stop_words = set(stopwords.words(langDict[language])) 
        word_tokens = word_tokenize(sentence)
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words and w not in additional_stopwords and len(w)>=3:
                filtered_sentence.append(w);
    #FIND Tweets without sentiment and place it into a text file per language
    if pd.isnull(twitter._source[i].get('sentiment')):
        TwitterWithout.append(twitter.iloc[[i]])
        try:
            filtered_sentence=' '.join(filtered_sentence)
        except ValueError:
            print('One-word Tweet')  
        fNeu.write(str(filtered_sentence)+"\n");
    #FIND Tweets with sentiment and place it into a text file per language
    else:
        #FIND Tweet with POSITIVE sentiment
        if twitter._source[i].get('sentiment')=='POSITIVE':
            TwitterWithPos.append(twitter.iloc[[i]])
            try:
                filtered_sentence=' '.join(filtered_sentence)
            except ValueError:
                print('One-word Tweet') 
            fPos.write(str(filtered_sentence)+"\n");
        #FIND Tweet with NEGATIVE sentiment
        elif twitter._source[i].get('sentiment')=='NEGATIVE':
            TwitterWithNeg.append(twitter.iloc[[i]])
            try:
                filtered_sentence=' '.join(filtered_sentence)
            except ValueError:
                print('One-word Tweet') 
            fNeg.write(str(filtered_sentence)+"\n");

#To transform into DataFrame       
TwitterWithPos=pd.concat(TwitterWithPos, ignore_index=True)
TwitterWithNeg=pd.concat(TwitterWithNeg, ignore_index=True)
TwitterWithout=pd.concat(TwitterWithout, ignore_index=True)

fPos.close()
fNeg.close()
fNeu.close()

# Last work to classify

Last problem still occuring:

1) I have no clue how to get all the word features in "extract_features" function. Problem linked to the fact we're using: nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_train) 
    
    nltk.classify.apply_features(function, toks)
    
    [function(tok) for tok in toks]
    
2) Problem to compute the accuracy. It worked at some point but could'nt fix it back with all the changes I added.

In [26]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from nltk.stem import PorterStemmer
import time
start_time = time.time()

def filtering():
    sentence=twitter.iloc[i]["_source"].get('main');
    word_tokens = word_tokenize(sentence)
    words_filtered=[x.lower() for x in word_tokens if x not in stop_words if x not in additional_stopwords if len(x)>=3]
    return words_filtered

def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features


additional_stopwords=['.',',','/','(',')','[',']',':','@','\'','?','!','\"',';','&']
TwitterWithPos=[]
TwitterWithNeg=[]
TwitterWithout=[]
all_filtered_sentence_en=[]
all_filtered_sentence_de=[]
all_filtered_sentence_fr=[]
all_filtered_sentence_it=[]

twitter['language'] = pd.Series(np.zeros(len(twitter)));

for i in range(0,400): #TO CHANGE !!! len(twitter)
    
    #ADD COLUMN TO ADD THE LANGUAGE
    sentence=twitter.iloc[i]["_source"].get('main')
    language='None';
    try:
        language=detect(sentence)
    except:
        e=1;
        
    twitter.ix[i,'language']=language
    
    #FIND the language of the current Tweet and create tuple (words of sentence, sentiment)
    if language=='en' or language=='de' or language=='it' or language=='fr': 
        langDict = {'en': 'english', 'de': 'german', 'it':'italian', 'fr':'french'} 
        stop_words = set(stopwords.words(langDict[language])) 
        
        if language=='en':
            words_filtered=filtering()
            all_filtered_sentence_en.append((words_filtered,twitter._source[i].get('sentiment'))) 
         
        elif language=='fr':
            words_filtered=filtering()
            all_filtered_sentence_fr.append((words_filtered,twitter._source[i].get('sentiment')))

        elif language=='de':
            words_filtered=filtering()
            all_filtered_sentence_de.append((words_filtered,twitter._source[i].get('sentiment')))
            
        elif language=='it':
            words_filtered=filtering()
            all_filtered_sentence_it.append((words_filtered,twitter._source[i].get('sentiment')))                        
        
        
        
    #DISSOCIATE ROW WITH/WITHOU SENTIMENT     
    #FIND Tweets without sentiment and place it into a text file per language
    if pd.isnull(twitter._source[i].get('sentiment')):
        TwitterWithout.append(twitter.iloc[[i]])
    #FIND Tweets with sentiment and place it into a text file per language
    else:
        #FIND Tweet with POSITIVE sentiment
        if twitter._source[i].get('sentiment')=='POSITIVE':
            TwitterWithPos.append(twitter.iloc[[i]]) 
        #FIND Tweet with NEGATIVE sentiment
        elif twitter._source[i].get('sentiment')=='NEGATIVE':
            TwitterWithNeg.append(twitter.iloc[[i]])

#CLASSIFYING

#word_features_en = get_word_features(get_words_in_tweets(all_filtered_sentence_en))
all_filtered_sentence_en_train=all_filtered_sentence_en[:round(0.8*len(all_filtered_sentence_en))]
training_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_train)
classifier_en = nltk.NaiveBayesClassifier.train(training_set_en)

#word_features_fr = get_word_features(get_words_in_tweets(all_filtered_sentence_fr))
all_filtered_sentence_fr_train=all_filtered_sentence_fr[:round(0.8*len(all_filtered_sentence_fr))]
training_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_train)
classifier_fr = nltk.NaiveBayesClassifier.train(training_set_fr)

#word_features_de = get_word_features(get_words_in_tweets(all_filtered_sentence_de))
all_filtered_sentence_de_train=all_filtered_sentence_de[:round(0.8*len(all_filtered_sentence_de))]
training_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_train)
classifier_de = nltk.NaiveBayesClassifier.train(training_set_de)

#word_features_it = get_word_features(get_words_in_tweets(all_filtered_sentence_it))
all_filtered_sentence_it_train=all_filtered_sentence_it[:round(0.8*len(all_filtered_sentence_it))]
training_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_train)
classifier_it = nltk.NaiveBayesClassifier.train(training_set_it)


#PRINT RESULTS
all_filtered_sentence_en_test=all_filtered_sentence_en[round(0.8*len(all_filtered_sentence_en)):]
testing_set_en = nltk.classify.apply_features(extract_features, all_filtered_sentence_en_test)
print(nltk.classify.accuracy(classifier_en, testing_set_en)*100)

all_filtered_sentence_fr_test=all_filtered_sentence_fr[round(0.8*len(all_filtered_sentence_fr)):]
testing_set_fr = nltk.classify.apply_features(extract_features, all_filtered_sentence_fr_test)
print(nltk.classify.accuracy(classifier_fr, testing_set_fr)*100)

all_filtered_sentence_de_test=all_filtered_sentence_de[round(0.8*len(all_filtered_sentence_de)):]
testing_set_de = nltk.classify.apply_features(extract_features, all_filtered_sentence_de_test)
print(nltk.classify.accuracy(classifier_de, testing_set_de)*100)

all_filtered_sentence_it_test=all_filtered_sentence_it[round(0.8*len(all_filtered_sentence_it)):]
testing_set_it = nltk.classify.apply_features(extract_features, all_filtered_sentence_it_test)
print(nltk.classify.accuracy(classifier_it, testing_set_it)*100)

#########

#To transform into DataFrame       
TwitterWithPos=pd.concat(TwitterWithPos, ignore_index=True)
TwitterWithNeg=pd.concat(TwitterWithNeg, ignore_index=True)
TwitterWithout=pd.concat(TwitterWithout, ignore_index=True)

print("--- %s seconds ---" % (time.time() - start_time))


NameError: name 'word_features' is not defined